In [9]:
import sys
import os

module_path = os.path.join(os.path.abspath(os.getcwd()))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# create dataset
!python3 src/dataset_creator.py --city --n_samples 20 --n_tasks 4 --out_file data/test.pkl

In [6]:
from functools import partial
from pathlib import Path

import numpy as np
import torch
from tqdm.auto import tqdm
from matplotlib.animation import FuncAnimation
from torch.utils.tensorboard import SummaryWriter

from src.VSPSolver import solve_vsp
from src.utils import get_model, get_criterion, get_optimizer, get_dataloaders
from src.city import City, SimpleDirectedGraph
from src.visualization_utils import visualize_paths


# add simple tensorboard to the trainer to visualize results 
class InteractiveTrainer:
    def __init__(self, config, log=False, **kwargs):
        self.config = config
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.model = get_model(config, device=self.device)
        self.train_loader, self.val_loader, self.test_loader = get_dataloaders(config)

        optimizer_class, optimizer_kwargs = get_optimizer(config)
        self.optimizer = optimizer_class(self.model.parameters(), **optimizer_kwargs)

        criterion_class, criterion_kwargs = get_criterion(config)
        self.criterion = lambda func: criterion_class(func, **criterion_kwargs)

        self.n_epochs = config["train"]["n_epochs"]
        self.eval_every = config["train"]["eval_every_n_epochs"]
        self.save_every = config["train"]["save_every_n_epochs"]
        self.save_dir = Path(config["train"]["save_dir"])
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.with_city = config["data"]["city"]
        self.writer = SummaryWriter(kwargs["logdir"]) if log else None

    # compare all paths to chosen path and SVSP path
    def update_log(self, instance, iter: int, loss: float, solution: list[int], labels: list[int], **kwargs):
        if isinstance(instance, City):
            figure = visualize_paths(instance, solution)
            self.writer.add_figure(f"City/city instance at iter {iter}", figure)
        self.writer.add_scalar(f"Loss/Loss at iter {iter}", loss, iter)
        self.writer.add_text(f"Desc", "what is this")

    def compute_metrics(self, i):
        if i % self.eval_every != 0:
            return
        losses = []
        self.model.eval()
        with torch.no_grad():
            for inputs, labels, instance in self.val_loader:
                graph = instance.graph if self.with_city else instance
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                theta = self.model(inputs)
                func = partial(solve_vsp, graph=graph)
                criterion = self.criterion(func)
                loss = criterion(theta, labels).mean()
                losses.append(loss.item())

        print(f"Validation loss: {np.mean(losses):.3f}")

    def save_model(self, i):
        if i % self.save_every == 0 and i > 0:
            torch.save(self.model.state_dict(), self.save_dir / f"epoch{i}.pt")

    def train_epoch(self, i):
        self.model.train()
        losses = []
        for iter, (inputs, labels, instance) in tqdm(enumerate(self.train_loader), desc=f"Epoch {i}"):
            graph = instance.graph if self.with_city else instance
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)

            self.optimizer.zero_grad()
            theta = self.model(inputs)

            func = partial(solve_vsp, graph=graph)
            criterion = self.criterion(func)
            loss = criterion(theta, labels)
            losses.append(loss.item())

            # workaround to get single solution from the solver
            solution = func(theta.detach().clone().unsqueeze(0))[0]

            loss.backward()
            self.optimizer.step()

            # visual logging here
            self.update_log(instance, iter, loss, solution, labels)

        print(f"Train loss: {np.mean(losses):.3f}")

    def train(self):
        for i in range(self.n_epochs):
            self.train_epoch(i)
            self.compute_metrics(i)
            self.save_model(i)

    def test(self):
        self.model.eval()
        losses = []
        with torch.no_grad():
            for inputs, labels, instance in tqdm(self.test_loader):
                graph = instance.graph if self.with_city else instance
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                theta = self.model(inputs)
                func = partial(solve_vsp, graph=graph)
                criterion = self.criterion(func)
                loss = criterion(theta, labels)
                losses.append(loss.item())

        print(f"Test loss: {np.mean(losses):.3f}")


In [ ]:
import yaml

with open("configs/test.yaml", "r") as file:
    config = yaml.safe_load(file)
    trainer = InteractiveTrainer(config, log=True, logdir="runs")
    trainer.train()
    trainer.test()